In [38]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import hydra
import glob
import pickle
from src.utils import get_cfg_from_from_ckpt_path
from sklearn.decomposition import PCA
import numpy as np
import re
from tqdm import tqdm
from src.utils import *
from src.model.autoencoder.AE_CNN_3D import AE_CNN_3D
from src.model.autoencoder.AE_CNN_1D import AE_CNN_1D
from src.explicit_ecs import ECS_explicit_pred_1D, ECS_explicit_pred_3D

import shutil


In [3]:
plt.style.use('ggplot')

In [4]:
cmap = plt.get_cmap('viridis')

In [5]:
load_data = True

In [6]:
gpu = 2
    
if torch.cuda.is_available() and gpu is not None:
##This may not be necessary outside the notebook
    dev = f"cuda:{gpu}"
else:
    dev = "cpu"

device = torch.device(dev)

# Utils Functions 

In [7]:
def load_model(model_ckpt_path: str,
               device: str):

    cfg = get_cfg_from_from_ckpt_path(model_ckpt_path, pprint = False)
    
    lit_mod = hydra.utils.call(cfg.model)

    lit_mod.load_state_dict(torch.load(model_ckpt_path)["state_dict"])


    lit_mod = lit_mod.to(device) # Move model to gpu for faster inference
    lit_mod = lit_mod.eval() # Model in eval mode
    for param in lit_mod.parameters():
        param.requires_grad = False  # Ensure no gradients are calculated for this model

    return lit_mod

# load data

In [7]:
ssf_da_path = "/DATASET/eNATL/eNATL60_BLB002_sound_speed_regrid_0_1000m.nc"
ssp_da = xr.open_dataarray(ssf_da_path).dropna(dim="lat")

coords = ssp_da.coords
depth_array = coords['z'].data


ssp_arr = ssp_da.data.transpose(0,2,3,1)
ssp_shape = ssp_arr.shape
#ssp_arr_flatten = np.flatten(ssp_arr)


# Load model

In [32]:
ckpt_path = "/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE_V2/AE_CNN_3D/pred_1_grad_0/channels_list_[1, 1]_1_conv_per_layer_padding_linear_pooling_None_final_upsample_upsample_pooling/act_fn_None_final_act_fn_None_normalization_min_max/manage_nan_suppress/n_profiles_None/lr_0.1/2024-09-09_20-31/checkpoints/val_loss=0.00-epoch=883.ckpt"
print(ckpt_path)

/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE_V2/AE_CNN_3D/pred_1_grad_0/channels_list_[1, 1]_1_conv_per_layer_padding_linear_pooling_None_final_upsample_upsample_pooling/act_fn_None_final_act_fn_None_normalization_min_max/manage_nan_suppress/n_profiles_None/lr_0.1/2024-09-09_20-31/checkpoints/val_loss=0.00-epoch=883.ckpt


In [33]:
cfg = get_cfg_from_from_ckpt_path(ckpt_path, pprint = True)

```yaml

datamodule:
  _target_: src.autoencoder_datamodule.AutoEncoderDatamodule_${dimensions}
  dl_kw: {batch_size: '${model_config.batch_size.${dimensions}}', num_workers: 4}
  dtype_str: ${dtype}
  input_da: {_target_: src.utils.load_ssp_da, ssf_da_path: '${paths.sound}'}
  manage_nan: suppress
  n_profiles: null
  norm_stats: {method: min_max, params: '${normalization.${datamodule.norm_stats.method}}'}
dimensions: 3D
dtype: float32
entrypoints:
- {_target_: pytorch_lightning.seed_everything, seed: 333}
- {_target_: src.train.base_training, dim: '${dimensions}', dm: '${datamodule}', lit_mod: '${model}',
  trainer: '${trainer}'}
model:
  _target_: src.autoencoder_V2.AutoEncoder
  depth_array: {_target_: src.utils.get_depth_array, ssf_da_path: '${paths.sound}'}
  dim: ${dimensions}
  loss_weight: {ecs_weight: 0, gradient_weight: 0, max_position_weight: 0, prediction_weight: 1,
    variation_weight: 0}
  model_hparams: ${model_config.model_hparams.${model.model_name}}
  model_name: ${model_architecture}_${dimensions}
  opt_fn: {T_max: 100000, _partial_: true, _target_: src.ecs_classification.ECS_classification.cosanneal_lr_adamw,
    lr: 0.1, weight_decay: 0}
model_architecture: AE_CNN
model_config:
  accumulate_grad_batches: {1D: 1, 2D: 1, 3D: 1}
  batch_size: {1D: 512, 2D: 32, 3D: 4}
  model_hparams:
    AE_CNN_1D:
      act_fn_str: None
      channels_list: [1, 1]
      dropout_proba: 0
      dtype_str: ${dtype}
      final_act_fn_str: None
      final_upsample_str: upsample_pooling
      latent_size: 9
      linear_layer: false
      n_conv_per_layer: 1
      padding: linear
      pooling: None
    AE_CNN_2D:
      act_fn_str: None
      channels_list: [1, 1]
      dropout_proba: 0
      dtype_str: ${dtype}
      final_act_fn_str: None
      final_upsample_str: upsample_pooling
      latent_size: 100
      linear_layer: false
      n_conv_per_layer: 1
      padding: linear
      pooling: None
    AE_CNN_3D:
      act_fn_str: None
      channels_list: [1, 1]
      dropout_proba: 0
      dtype_str: ${dtype}
      final_act_fn_str: None
      final_upsample_str: upsample_pooling
      latent_size: 100
      linear_layer: false
      n_conv_per_layer: 1
      padding: linear
      pooling: None
    AE_CNN_pool_2D:
      act_fn_str: Sigmoid
      dropout_proba: 0
      dtype_str: float32
      final_act_fn_str: Relu
      init_params:
        params: {_target_: src.utils.get_convo_init_weight_bias, init_params_pickle_path: "/homes/o23gauvr/Documents/th\xE8\
            se/code/FASCINATION/pickle/pca_mean_and_components.pkl"}
        use: false
      input_channels: 107
      latent_dim: 9
      num_layers: 1
      pooling_str: Max
  save_dir: {AE_CNN: 'channels_list_${model.model_hparams.channels_list}_${model.model_hparams.n_conv_per_layer}_conv_per_layer_padding_${model.model_hparams.padding}_pooling_${model.model_hparams.pooling}_final_upsample_${model.model_hparams.final_upsample_str}/act_fn_${model.model_hparams.act_fn_str}_final_act_fn_${model.model_hparams.final_act_fn_str}_normalization_${datamodule.norm_stats.method}',
    AE_CNN_pool_1D: 'latent_dim_${model.model_hparams.latent_dim}_pooling_${model.model_hparams.num_layers}_${model.model_hparams.pooling_str}_dropout_${model.model_hparams.dropout_proba}_patience_${patience}',
    AE_CNN_pool_2D: 'latent_dim_${model.model_hparams.latent_dim}_pooling_${model.model_hparams.num_layers}_${model.model_hparams.pooling_str}_dropout_${model.model_hparams.dropout_proba}_patience_${patience}/act_fn_${model.model_hparams.act_fn_str}_final_act_fn_${model.model_hparams.final_act_fn_str}_normalization_${datamodule.norm_stats.method}'}
normalization:
  mean_std: {mean: null, std: null}
  mean_std_along_depth: {mean: null, std: null}
  min_max: {x_max: null, x_min: null}
paths: {sound: /DATASET/eNATL/eNATL60_BLB002_sound_speed_regrid_0_1000m.nc}
patience: 25
trainer:
  _target_: pytorch_lightning.Trainer
  accelerator: gpu
  accumulate_grad_batches: ${model_config.accumulate_grad_batches.${dimensions}}
  callbacks:
  - {_target_: pytorch_lightning.callbacks.LearningRateMonitor}
  - {_target_: pytorch_lightning.callbacks.ModelCheckpoint, filename: '{val_loss:.2f}-{epoch:02d}',
    mode: min, monitor: val_loss, save_top_k: 1}
  - {_target_: pytorch_lightning.callbacks.EarlyStopping, check_on_train_epoch_end: true,
    min_delta: 0.0, monitor: val_loss, patience: '${trainer.max_epochs}', verbose: true}
  check_val_every_n_epoch: 1
  devices: 1
  inference_mode: false
  logger: {_target_: pytorch_lightning.loggers.tensorboard.TensorBoardLogger, log_graph: false,
    name: '${model.model_name}/pred_${model.loss_weight.prediction_weight}_grad_${model.loss_weight.gradient_weight}',
    save_dir: "/homes/o23gauvr/Documents/th\xE8se/code/FASCINATION/outputs/AE_V2/",
    version: '${model_config.save_dir.${model_architecture}}/manage_nan_${datamodule.manage_nan}/n_profiles_${datamodule.n_profiles}/lr_${model.opt_fn.lr}/${now:%Y-%m-%d_%H-%M}'}
  max_epochs: 2000
  min_epochs: 0


```

In [34]:
lit_model = load_model(ckpt_path, device)

# Set up data

In [35]:
dm = hydra.utils.call(cfg.datamodule) # will instantiate src.data.BaseDataModule with parameters specified in config
dm.setup(stage="fit") # setup the datamodule see https://lightning.ai/docs/pytorch/stable/data/datamodule.html#lightningdatamodule-api
dm.setup(stage="test") # setup the datamodule see https://lightning.ai/docs/pytorch/stable/data/datamodule.html#lightningdatamodule-api

In [36]:
train_ssp_arr = dm.train_dataloader().dataset.input.data
train_ssp_tens = torch.tensor(train_ssp_arr).float().to(device)
input_train_shape = train_ssp_arr.shape

test_ssp_arr = dm.test_dataloader().dataset.input.data
test_ssp_tens = torch.tensor(test_ssp_arr).float().to(device)
input_test_shape = test_ssp_arr.shape

In [37]:
mse = []
for ssp_truth in (dm.train_dataloader()):
    ssp_truth = ssp_truth.to(lit_model.device) 
    ssp_pred = lit_model(ssp_truth.to(lit_model.device))
    mse.append(torch.mean((ssp_truth-ssp_pred)**2).item())
    
    

In [38]:
np.sort(mse)

array([1.31574134e-05, 1.32356190e-05, 1.32518153e-05, 1.34056017e-05,
       1.36985464e-05, 1.37362968e-05, 1.37428615e-05, 1.39450131e-05,
       1.39596823e-05, 1.40508910e-05, 1.41116461e-05, 1.41176024e-05,
       1.41349055e-05, 1.42970939e-05, 1.43207317e-05, 1.43642810e-05,
       1.44026335e-05, 1.44498335e-05, 1.45986069e-05, 1.47260980e-05,
       1.47356695e-05, 1.47371420e-05, 1.47570163e-05, 1.47587207e-05,
       1.48349372e-05, 1.48664067e-05, 1.49224943e-05, 1.50545447e-05,
       1.51686654e-05, 1.52205248e-05, 1.52324446e-05, 1.53139717e-05,
       1.53965520e-05, 1.54155659e-05, 1.54283061e-05, 1.54893460e-05,
       1.55171547e-05, 1.55327052e-05, 1.55919515e-05, 1.56767837e-05,
       1.57195755e-05, 1.57619379e-05, 1.59694955e-05, 1.60573090e-05,
       1.60773689e-05, 1.61378375e-05, 1.62164251e-05, 1.62694341e-05,
       1.65364454e-05, 1.65841084e-05, 1.65953916e-05, 1.66125647e-05,
       1.66314876e-05, 1.67130329e-05, 1.67509697e-05, 1.68021652e-05,
      

In [39]:
np.max(mse)

1.9154880646965466e-05

# Move file

In [63]:
"[1,1]_trilinear_test_on_padding" in "/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE/AE_CNN_3D/[1,1]_trilinear_test_on_padding/pred_1_grad_0/upsample_mode_trilinear/1_conv_per_layer/padding_linear/pooling_Max_on_dim_all/final_upsample_upsample_pooling/act_fn_None_final_act_fn_None/lr_0.001/normalization_min_max/manage_nan_suppress/n_profiles_None/2024-10-10_15-00/checkpoints/val_loss=0.00-epoch=555.ckpt"

True

In [64]:
ckpt_list

[PosixPath('/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE/AE_CNN_3D/channels_[1, 1]/pred_1_grad_0/channels_list_[1, 1]_1_conv_per_layer_padding_linear_pooling_Max_on_dim_all_final_upsample_upsample_pooling/act_fn_None_final_act_fn_Sigmoid_normalization_min_max/manage_nan_suppress/n_profiles_None/lr_0.0001/2024-09-23_13-40/checkpoints/val_loss=0.00-epoch=182.ckpt')]

In [65]:
output_path = "/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs"

dir_to_ignore = ["[1,1]_trilinear_test_on_padding", "upsample_mode_test_on_[1,1]"]

ckpt_list = list(Path(output_path).rglob('*.ckpt'))

for ckpt_path in tqdm(ckpt_list):
    
    ckpt_path = str(ckpt_path)
    
    if any(dir_name in ckpt_path for dir_name in dir_to_ignore):
        print(f"Skipping {ckpt_path} because it contains a directory to ignore.")
        continue
    

100%|██████████| 171/171 [00:00<00:00, 246554.14it/s]

Skipping /homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE/AE_CNN_3D/upsample_mode_test_on_[1,1]/pred_1_grad_0/upsample_mode_trilinear/1_conv_per_layer/padding_linear/pooling_Max_on_dim_all/final_upsample_upsample_pooling/act_fn_None_final_act_fn_None/lr_0.001/normalization_min_max/manage_nan_suppress/n_profiles_None/2024-10-09_17-00/checkpoints/val_loss=0.00-epoch=783.ckpt because it contains a directory to ignore.
Skipping /homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE/AE_CNN_3D/upsample_mode_test_on_[1,1]/pred_1_grad_0/upsample_mode_area/1_conv_per_layer/padding_linear/pooling_Max_on_dim_all/final_upsample_upsample_pooling/act_fn_None_final_act_fn_None/lr_0.001/normalization_min_max/manage_nan_suppress/n_profiles_None/2024-10-10_14-23/checkpoints/val_loss=0.00-epoch=661.ckpt because it contains a directory to ignore.
Skipping /homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE/AE_CNN_3D/[1,1]_trilinear_test_on_padding/pred_1_grad_0/upsample_mode_trilin

In [22]:
cfg = get_cfg_from_from_ckpt_path(str(ckpt_path), pprint = False)

channels_${model.model_hparams.channels_list}/pred_${model.loss_weight.prediction_weight}_grad_${model.loss_weight.gradient_weight}/${model.model_hparams.n_conv_per_layer}_conv_per_layer/padding_${model.model_hparams.padding}/pooling_${model.model_hparams.pooling}_on_dim_${model.model_hparams.pooling_dim}/final_upsample_${model.model_hparams.final_upsample_str}/act_fn_${model.model_hparams.act_fn_str}_final_act_fn_${model.model_hparams.final_act_fn_str}/lr_${model.opt_fn.lr}/normalization_${datamodule.norm_stats.method}/manage_nan_${datamodule.manage_nan}/n_profiles_${datamodule.n_profiles}/${now:%Y-%m-%d_%H-%M}

In [56]:

dest_dir =  f"{cfg["trainer"]["logger"]["save_dir"].split("AE")[0]}AE/" \
            f"{cfg['model']['model_name']}" \
            f"channels_{channels_list}/pred_{prediction_weight}_grad_{gradient_weight}/" \
            f"{n_conv_per_layer}_conv_per_layer/padding_{padding}/pooling_{pooling}_on_dim_{pooling_dim}/" \
            f"final_upsample_{final_upsample_str}/act_fn_{act_fn_str}_final_act_fn_{final_act_fn_str}/" \
            f"lr_{lr}/normalization_{normalization_method}/manage_nan_{manage_nan}/n_profiles_{n_profiles}/{date}"
dest_dir

SyntaxError: f-string: unmatched '[' (2721036059.py, line 1)

In [ ]:
/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE/ ${model.model_name}

In [59]:
output_path = cfg["trainer"]["logger"]["save_dir"].split("AE")[0]
model_name = cfg['model']['model_name']

channels_list = str(cfg['model_config']['model_hparams']['AE_CNN_3D']['channels_list'])
prediction_weight = cfg['model']['loss_weight']['prediction_weight']
gradient_weight = cfg['model']['loss_weight']['gradient_weight']
n_conv_per_layer = cfg['model_config']['model_hparams']['AE_CNN_3D']['n_conv_per_layer']
padding = cfg['model_config']['model_hparams']['AE_CNN_3D']['padding']
pooling = cfg['model_config']['model_hparams']['AE_CNN_3D']['pooling']
pooling_dim = cfg['model_config']['model_hparams']['AE_CNN_3D']['pooling_dim']
final_upsample_str = cfg['model_config']['model_hparams']['AE_CNN_3D']['final_upsample_str']
act_fn_str = cfg['model_config']['model_hparams']['AE_CNN_3D']['act_fn_str']
final_act_fn_str = cfg['model_config']['model_hparams']['AE_CNN_3D']['final_act_fn_str']
lr = cfg['model']['opt_fn']['lr']
normalization_method = cfg['datamodule']['norm_stats']['method']
manage_nan = cfg['datamodule']['manage_nan']
n_profiles = cfg['datamodule']['n_profiles']


date = str(ckpt_path).split("/")[-3]



dir_to_relocate = "/".join(str(ckpt_path).split("/")[:-2])


dest_dir =  f"{output_path}AE/" \
            f"{model_name}/" \
            f"channels_{channels_list}/pred_{prediction_weight}_grad_{gradient_weight}/" \
            f"{n_conv_per_layer}_conv_per_layer/padding_{padding}/pooling_{pooling}_on_dim_{pooling_dim}/" \
            f"final_upsample_{final_upsample_str}/act_fn_{act_fn_str}_final_act_fn_{final_act_fn_str}/" \
            f"lr_{lr}/normalization_{normalization_method}/manage_nan_{manage_nan}/n_profiles_{n_profiles}/{date}"


In [60]:
dest_dir

'/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE/AE_CNN_3D/channels_[1, 2, 4, 8, 16]/pred_1_grad_100000/1_conv_per_layer/padding_linear/pooling_Max_on_dim_all/final_upsample_upsample_pooling/act_fn_None_final_act_fn_None/lr_0.001/normalization_min_max/manage_nan_suppress/n_profiles_None/2024-10-05_12-16'

In [34]:
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

In [40]:
for item in os.listdir(dir_to_relocate):
    src = os.path.join(dir_to_relocate, item)
    dest = os.path.join(dest_dir, item)
    shutil.move(src, dest)